# KServe Pipeline samples

This notebook assumes your cluster has KServe >= v0.7.0 installed which supports the v1beta1 API.

### Install the necessary kfp library

In [ ]:
!pip3 install kfp --upgrade

In [ ]:
import kfp.compiler as compiler
import kfp.dsl as dsl
import kfp
from kfp import components

# Create kfp client
# Note: Add the KubeFlow Pipeline endpoint below if the client is not running on the same cluster.
# Example: kfp.Client('http://192.168.1.27:31380/pipeline')
client = kfp.Client()
EXPERIMENT_NAME = 'KServe Experiments'
experiment = client.create_experiment(name=EXPERIMENT_NAME, namespace='anonymous')

### TensorFlow example

In [ ]:
# kfserving_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')
kserve_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kserve/component.yaml')

@dsl.pipeline(
  name='KServe pipeline',
  description='A pipeline for KServe.'
)
def kservePipeline(
    action='apply',
    model_name='tensorflow-sample',
    model_uri='gs://kfserving-examples/models/tensorflow/flowers',
    namespace='anonymous',
    framework='tensorflow'):

    kserve = kserve_op(action = action,
                             model_name=model_name,
                             model_uri=model_uri,
                             namespace=namespace,
                             framework=framework).set_image_pull_policy('Always')

# Compile pipeline
compiler.Compiler().compile(kservePipeline, 'tf-flower.tar.gz')

# Execute pipeline
run = client.run_pipeline(experiment.id, 'tf-flower', 'tf-flower.tar.gz')

### Custom model example

In [ ]:
# kfserving_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')
kserve_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kserve/component.yaml')

@dsl.pipeline(
  name='KServe pipeline',
  description='A pipeline for KServe.'
)
def kservePipeline(
    action='apply',
    model_name='max-image-segmenter',
    namespace='anonymous',
    custom_model_spec='{"name": "image-segmenter", "image": "codait/max-image-segmenter:latest", "port": "5000"}'
):

    kserve = kserve_op(action=action,
                             model_name=model_name,
                             namespace=namespace,
                             custom_model_spec=custom_model_spec).set_image_pull_policy('Always')

# Compile pipeline
compiler.Compiler().compile(kservePipeline, 'custom.tar.gz')

# Execute pipeline
run = client.run_pipeline(experiment.id, 'custom-model', 'custom.tar.gz')